## #  Ubiquant Market Prediction with DNN
## Import Packages

In [ ]:
import gc
gc.collect()

from pathlib import Path
from argparse import Namespace
from collections import defaultdict


In [ ]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from scipy import stats

## Import dataset

In [ ]:
#%%time
n_features = 300
features = [f'f_{i}' for i in range(n_features)]
train = pd.read_pickle('../input/ubiquant-market-prediction-half-precision-pickle/train.pkl')
train.head()

In [ ]:
gc.collect()


In [ ]:
investment_id = train.pop("investment_id")
investment_id.head()

In [ ]:
_ = train.pop("time_id")

In [ ]:
y = train.pop("target")
y.head()

# Define Score&Loss function

In [ ]:
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops

def t(a):
  """For testing: generate a float64 tensor from anything."""
  return tf.constant(a, dtype=tf.float64)

def tmean(x, axis=-1):
  """Arithmetic mean of a tensor over some axis, default last."""
  x = tf.convert_to_tensor(x)
  sum = tf.reduce_sum(x, axis=axis)
  n = tf.cast(tf.shape(x)[axis], x.dtype)
  return sum / n

def correlationMetric(x, y, axis=-2):
  """Metric returning the Pearson correlation coefficient of two tensors over some axis, default -2."""
  x = tf.convert_to_tensor(x)
  y = math_ops.cast(y, x.dtype)
  n = tf.cast(tf.shape(x)[axis], x.dtype)
  xsum = tf.reduce_sum(x, axis=axis)
  ysum = tf.reduce_sum(y, axis=axis)
  xmean = xsum / n
  ymean = ysum / n
  xvar = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
  yvar = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
  cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
  corr = cov / tf.sqrt(xvar * yvar)
  return tf.constant(1.0, dtype=x.dtype) - corr


def correlationLoss(x,y, axis=-2):
  """Loss function that maximizes the pearson correlation coefficient between the predicted values and the labels,
  while trying to have the same mean and variance"""
  x = tf.convert_to_tensor(x)
  y = math_ops.cast(y, x.dtype)
  n = tf.cast(tf.shape(x)[axis], x.dtype)
  xsum = tf.reduce_sum(x, axis=axis)
  ysum = tf.reduce_sum(y, axis=axis)
  xmean = xsum / n
  ymean = ysum / n
  xsqsum = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
  ysqsum = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
  cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
  corr = cov / tf.sqrt(xsqsum * ysqsum)
  # absdif = tmean(tf.abs(x - y), axis=axis) / tf.sqrt(yvar)
  sqdif = tf.reduce_sum(tf.math.squared_difference(x, y), axis=axis) / n / tf.sqrt(ysqsum / n)
  # meandif = tf.abs(xmean - ymean) / tf.abs(ymean)
  # vardif = tf.abs(xvar - yvar) / yvar
  # return tf.convert_to_tensor( K.mean(tf.constant(1.0, dtype=x.dtype) - corr + (meandif * 0.01) + (vardif * 0.01)) , dtype=tf.float32 )
  return tf.convert_to_tensor( K.mean(tf.constant(1.0, dtype=x.dtype) - corr + (0.01 * sqdif)) , dtype=tf.float32 )

## Create a IntegerLookup layer for investment_id input

In [ ]:
#%%time
investment_ids = list(investment_id.unique())
investment_id_size = len(investment_ids) + 1
investment_id_lookup_layer = layers.IntegerLookup(max_tokens=investment_id_size)
investment_id_lookup_layer.adapt(pd.DataFrame({"investment_ids":investment_ids}))

## Make Tensorflow dataset

In [ ]:
def gen(feature, investment_id, y):
    for i in range(len(investment_id)):
        yield (investment_id.iloc[i], feature.iloc[i]), y.iloc[i]

In [ ]:
# t=gen(train[0:10],investment_id[0:10], y[0:10])
# t.__next__()

In [ ]:
# print(len(train),len(investment_id),len(y))
# print(train.shape,investment_id.shape,y.shape)
# for i in range(len(investment_id[0:10])):
    
#     print((investment_id.iloc[i], train.iloc[i]), y.iloc[i])

In [ ]:
# import tensorflow as tf
# def preprocess(X, y):
#     return X, y
# def make_dataset(feature, investment_id, y, batch_size=1024, mode="train"):
#     ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature), y))
#     ds = ds.map(preprocess)
#     if mode == "train":
#         ds = ds.shuffle(4096)
#     ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
#     return ds

In [ ]:
import tensorflow as tf
def preprocess(X, y):
    return X, y
def make_dataset(feature, investment_id, y, batch_size=1024, mode="train"):
    ds = tf.data.Dataset.from_generator(lambda: gen(feature, investment_id, y),
                                        output_types=((tf.uint16,tf.float16),tf.float16),
                                       output_shapes=(((),(300)),()))
    ds = ds.map(preprocess)
    if mode == "train":
        ds = ds.shuffle(1024)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [ ]:
#make_dataset(train[0:10],investment_id[0:10], y[0:10])

## Modeling

In [ ]:
# def get_model():
#     investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
#     features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
#     investment_id_x = investment_id_lookup_layer(investment_id_inputs)
#     investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
#     investment_id_x = layers.Reshape((-1, ))(investment_id_x)
#     investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)    
#     investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
#     investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
#     investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)   
    
#     feature_x = layers.Dense(256, activation='swish')(features_inputs)
#     feature_x = layers.Dense(256, activation='swish')(feature_x)
#     feature_x = layers.Dense(256, activation='swish')(feature_x)
#     feature_x = layers.Dense(256, activation='swish')(feature_x)
#     feature_x = layers.Dropout(0.65)(feature_x)
    
#     x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
#     x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
#     x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
#     x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
#     x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
#     x = layers.Dropout(0.75)(x)
#     output = layers.Dense(1)(x)
#     rmse = keras.metrics.RootMeanSquaredError(name="rmse")
#     model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
#     model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
#     return model

In [ ]:
def get_model():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(128, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    #investment_id_x = layers.Dense(32, activation='swish')(investment_id_x)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dense(128, activation='swish')(feature_x)
    #feature_x = layers.Dense(64, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(256, activation='swish', kernel_regularizer="l2")(x)
    # x = layers.Dense(256, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    #x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    # x = layers.Dense(16, activation='swish', kernel_regularizer="l2")(x)


    x = layers.BatchNormalization(name='batch_norm1')(x)
    x = layers.Dense(256, activation='swish', name='dense1')(x)
    x = layers.Dropout(0.1, name='dropout1')(x)
    x = layers.Reshape((1, -1), name='reshape1')(x)
    x = layers.BatchNormalization(name='batch_norm2')(x)
    x = layers.LSTM(128, return_sequences=True, activation='relu', name='lstm1')(x)
    x = layers.LSTM(16, return_sequences=False, activation='relu', name='lstm2')(x)


    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss=correlationLoss, metrics=['mae', correlationMetric])
    return model


In [ ]:
model = get_model()
model.summary()
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
gc.collect()

In [ ]:
#%%time
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit
#kfold = StratifiedKFold(5)
kfold =  TimeSeriesSplit(n_splits=10,max_train_size=2000000)

models = []

for index, (train_indices, valid_indices) in enumerate(kfold.split(train, investment_id)):
    if index<=5:
        
        X_train, X_val = train.iloc[train_indices], train.iloc[valid_indices]
        investment_id_train = investment_id[train_indices]
        y_train, y_val = y.iloc[train_indices], y.iloc[valid_indices]
        investment_id_val = investment_id[valid_indices]
        train_ds = make_dataset(X_train, investment_id_train, y_train)
        valid_ds = make_dataset(X_val, investment_id_val, y_val, mode="valid")
        model = get_model()
        checkpoint = keras.callbacks.ModelCheckpoint(f"model_{index}", save_best_only=True)
        #checkpoint = keras.callbacks.ModelCheckpoint(f"model_{index}.hdf5", save_best_only=True)

        early_stop = keras.callbacks.EarlyStopping(patience=5)
        #history = model.fit(train_ds, epochs=25,callbacks=[checkpoint,early_stop])
        history = model.fit(train_ds, epochs=20, validation_data=valid_ds,callbacks=[checkpoint,early_stop])
        #m_load=model.load_weights(f"model_{index}")
        #history = model.fit(train_ds, epochs=20, validation_data=valid_ds)
        m_load=keras.models.load_model(f"model_{index}",compile=False)
        #m_load_compile=m_load.compile(optimizer=tf.optimizers.Adam(0.001), loss=correlationLoss, metrics=['mae', correlationMetric])
        models.append(m_load)
        #models.append(history)

        pearson_score = stats.pearsonr(model.predict(valid_ds).ravel(), y_val.values)[0]
        print('Pearson:', pearson_score)
        #pd.DataFrame(history.history, columns=["mse", "val_mse"]).plot()
        #plt.title("MSE")
        #plt.show()
        #pd.DataFrame(history.history, columns=["mae", "val_mae"]).plot()
        #plt.title("MAE")
        #plt.show()
        #pd.DataFrame(history.history, columns=["rmse", "val_rmse"]).plot()
        #plt.title("RMSE")
        #plt.show()
        del investment_id_train
        del investment_id_val
        del X_train
        del X_val
        del y_train
        del y_val
        del train_ds
        del valid_ds
        del history
        del m_load
    else:
        break
    

    gc.collect()
    


In [ ]:
# i=[]
# t_indices=[]
# v_indices=[]


# for index, (train_indices, valid_indices) in enumerate(kfold.split(train, investment_id)):
#     i.append(index)
#     t_indices.append(train_indices)
#     v_indices.append(valid_indices)

# print(len(i),len(t_indices),len(v_indices))

In [ ]:

# X_train, X_val = train.iloc[t_indices[0]], train.iloc[v_indices[0]]
# investment_id_train = investment_id[t_indices[0]]
# y_train, y_val = y.iloc[t_indices[0]], y.iloc[v_indices[0]]
# investment_id_val = investment_id[v_indices[0]]
# train_ds = make_dataset(X_train, investment_id_train, y_train)
# valid_ds = make_dataset(X_val, investment_id_val, y_val, mode="valid")
# model = get_model()
# checkpoint = keras.callbacks.ModelCheckpoint(f"model_{i[0]}", save_best_only=True)
# #checkpoint = keras.callbacks.ModelCheckpoint(f"model_{index}.hdf5", save_best_only=True)
    
# early_stop = keras.callbacks.EarlyStopping(patience=5)
# history = model.fit(train_ds, epochs=20, validation_data=valid_ds,callbacks=[checkpoint,early_stop])
# #m_load=model.load_weights(f"model_{index}")
# #history = model.fit(train_ds, epochs=20, validation_data=valid_ds)
# m_load=keras.models.load_model(f"model_{i[0]}",compile=False)
# m_load_compile=m_load.compile(optimizer=tf.optimizers.Adam(0.001), loss=correlationLoss, metrics=['mae', correlationMetric])
# models.append(m_load_compile)
# #models.append(history)

In [ ]:
# pearson_score = stats.pearsonr(model.predict(valid_ds).ravel(), y_val.values)[0]
# print('Pearson:', pearson_score)

In [ ]:
def preprocess_test(investment_id, feature):
    return (investment_id, feature), 0
def make_test_dataset(feature, investment_id, batch_size=1024):
    ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature)))
    ds = ds.map(preprocess_test)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

def inference(models, ds):
    y_preds = []
    for model in models:
        y_pred = model.predict(ds)
        y_preds.append(y_pred)
    return np.mean(y_preds, axis=0)

In [ ]:
# import ubiquant
# env = ubiquant.make_env()
# iter_test = env.iter_test() 
# for (test_df, sample_prediction_df) in iter_test:
#     ds = make_test_dataset(test_df[features], test_df["investment_id"])
#     sample_prediction_df['target'] = inference(models, ds)
#     env.predict(sample_prediction_df) 
#     display(sample_prediction_df)

In [ ]:
len(models)